In [2]:
#Install all python packages with: pip install package
import ast
import time
import flopy
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
from PySAGA_cmd import SAGA
import whitebox
from shapely.geometry import Point, LineString, MultiPoint
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.colors import TwoSlopeNorm
from collections import defaultdict
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


#Link your GW results
#Input 1) Model Results, with name "modelo"and "version" 
modelo="Model_12" ### Change for your model name###
version= "12.136" ### Define your number model
epgs="EPSG:32719" ### change for your epgs number###
modelName = f'Model_v{version}'
modelWs = os.path.join(modelo, f"Sim_{version}")
model_path = os.path.join('C:/Users/nick_/OneDrive/Escritorio/Postgrado/Tesis/MODELMUSE',modelWs) #Cambiar a tu directorio donde estan inputs y outputs de mf6

#For Calibration with Springs
calib_manan_input=r'C:\Users\nick_\OneDrive\Escritorio\Raster'  #Change for your calibration_spring files path, already create with DEM input
calib_manan_path =os.path.join(calib_manan_input,'calib_output') #output
os.makedirs(calib_manan_path, exist_ok=True)
calib_man = f'Calib_Manan_Sim_{version}'

In [6]:
#help(function) if you have questions in whiteboxtools
wbt = whitebox.WhiteboxTools()
wbt.set_verbose_mode(True)

#INPUT Paths

#Input 2) DEM (for drainage network and d8_pointer)
#DEM with SRC ready for your targets, here is epgs:32719
DEM=os.path.join(calib_manan_input,"DEM.tif")

#Input 3) Shapefile of Dominium
dom=os.path.join(calib_manan_input,'Dominium.shp')

#Input 4) Shapefile of springs (points)
original_points = os.path.join(calib_manan_input,'Springs.shp')

#OUTPUTS Paths
DEM_OUT=os.path.join(calib_manan_path,"DEM_OUT.tif")
FP=os.path.join(calib_manan_path,"FP.tif")
FA=os.path.join(calib_manan_path,"FA.tif")
Stream_raster1=os.path.join(calib_manan_path,"STREAM.tif")
drainage_network=os.path.join(calib_manan_path,"drainage_network.shp")

#wbt work
wbt.flow_accumulation_full_workflow(dem=DEM,out_dem=DEM_OUT,out_pntr=FP,out_accum=FA,out_type='cells') #FA with cells

wbt.extract_streams(flow_accum=FA,output=Stream_raster1,threshold=1000) #Create Raster of drainage network with minimum threshold =1, trying with largest number, if you use 1 for a big regional DEM will take time

wbt.raster_streams_to_vector(Stream_raster1,FP,output=drainage_network) #Create Shape of drainage network

.\whitebox_tools.exe --run="FlowAccumulationFullWorkflow" --dem='C:\Users\nick_\OneDrive\Escritorio\Raster\DEM.tif' --out_dem='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\DEM_OUT.tif' --out_pntr='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\FP.tif' --out_accum='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\FA.tif' --out_type=cells -v --compress_rasters=False

*******************************************
* Welcome to FlowAccumulationFullWorkflow *
* Powered by WhiteboxTools                *
* www.whiteboxgeo.com                     *
*******************************************
Reading data...
Calculating aspect: 0%
Calculating aspect: 1%
Calculating aspect: 2%
Calculating aspect: 3%
Calculating aspect: 4%
Calculating aspect: 5%
Calculating aspect: 6%
Calculating aspect: 7%
Calculating aspect: 8%
Calculating aspect: 9%
Calculating aspect: 10%
Calculating aspect: 11%
Calculating aspect: 12%
Calculating aspect: 13%
Calculating aspect: 14%
Calculating aspect: 15

0

In [8]:
# Load GWT model and heads 
sim = flopy.mf6.MFSimulation.load(sim_name=modelName, sim_ws=model_path) 
gwf = sim.get_model()
head = gwf.output.head().get_data(kstpkper=(0, 0))

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package obs...
    loading package oc...
    loading package chd...
    loading package wel...
    loading package riv...
INFORMATION: maxbound in ('gwf6', 'riv', 'dimensions') changed to 18821 based on size of stress_period_data
    loading package drn...
    loading package rch...
    loading package hfb...
  loading solution package modflow...


In [9]:
#Get shape with all domain cells
# Get the idomain from the DIS package
idomain = gwf.dis.idomain.array  # Array 3D (layers, rows, columns)

# Extract active cells from the first layer (index 0)
layer = 0
rows, cols = np.where(idomain[layer] == 1)  # rows and columns activas

# Generar polígonos para cada celda activa
polygons = []
for row, col in zip(rows, cols):
    cell_vertices = gwf.modelgrid.get_cell_vertices(row, col)  # Get the vertices
    polygons.append(Polygon(cell_vertices))

# Create GeoDataFrame
gdf_domain = gpd.GeoDataFrame(geometry=polygons)
gdf_domain.crs = epgs  # Correct SRC

In [10]:
#Create folder to save files in the future
carpeta=os.path.join(calib_manan_path,calib_man)
os.makedirs(carpeta, exist_ok=True)
# Extract results of DRN package, for the steady state (layer, row, column, elev, cond)
drn_package = gwf.get_package("drn") 
drn_data = drn_package.stress_period_data.get_data()[0]
# CBC results
cbc = gwf.output.budget()
drn_cbc = cbc.get_data(text="DRN", kstpkper=(0, 0))[0]
# Diccionary for unique cells (row, col)
celdas_unicas = defaultdict(list)
# Group by row and column (without layer)
for cell in drn_data:
    cellid, elev_drn, cond, aux1, aux2, comment = cell  
    layer, row, col = cellid  
    celdas_unicas[(row, col)].append((layer, elev_drn))
# Create lists
polygons = []
elevaciones_drn = []
# Process each unique cell
for (row, col), valores in celdas_unicas.items():
    # Obtain the average elevation (or take the first one)
    elev_promedio = np.mean([v[1] for v in valores])
    
    # Get vertices of the cell
    cell_vertices = gwf.modelgrid.get_cell_vertices(row, col)
    polygon = Polygon(cell_vertices)
        
    polygons.append(polygon)
    elevaciones_drn.append(elev_promedio)
# Create GeoDataFrame
gdf_drn0 = gpd.GeoDataFrame(geometry=polygons,data={"elev_drn": elevaciones_drn})
gdf_drn0.crs = epgs 
# Now see the active cells in DRN package
celdas_agrupadas = defaultdict(list)
# Group DRN and CBC dates by cells, ignore layer
for cell, cbc_cell in zip(drn_data, drn_cbc):
    cellid, elev, cond, aux1, aux2, comment = cell
    layer, row, col = cellid
    q = cbc_cell["q"]
    
    # Save all values per cell (row, col)
    celdas_agrupadas[(row, col)].append({
        "layer": layer,
        "elev": elev,
        "q": q })                           
# Process each unique cell
polygons, elevaciones_drn, activos, cellids = [], [], [], []
for (row, col), valores in celdas_agrupadas.items():
    # Get cell geometry
    vertices = gwf.modelgrid.get_cell_vertices(row, col)
    polygon = Polygon(vertices)
    
    # Calculate values
    elev_promedio = np.mean([v["elev"] for v in valores])
    
    # Determine if it is active (if AT LEAST ONE input has q < 0)
    activo = any(v["q"] < 0 for v in valores)
        
    #Save Cellid
    cellid=(valores[0]["layer"], row, col)
    
    polygons.append(polygon)
    elevaciones_drn.append(elev_promedio)
    activos.append(activo) 
    cellids.append(cellid)
# Create GeoDataFrame
gdf_drn = gpd.GeoDataFrame(
    geometry=polygons,
    data={
        "elev_drn": elevaciones_drn,
        "activo": activos,
        "n_capas": [len(valores) for (row, col), valores in celdas_agrupadas.items()], 
        "cellid":cellids},
    crs="EPSG:32719")
gdf_drn_activas = gdf_drn[gdf_drn["activo"]].copy()


In [11]:
###If you already run this next cells once time, JUST RUN the next and the follow cell.

In [ ]:
#Define some Functions:

#To extract vertices
def extract_specific_vertices(gdf, indices=[0, -1]):
    """
    Extract specific vertices from the geometries of a GeoDataFrame.
    
    Parameters:
    - gdf: GeoDataFrame with LineString or MultiLineString geometries.
    - indices: List of indices of vertices to extract (e.g., [0, -1] for first and last).
    
    Returns:
    - GeoDataFrame with points representing the selected vertices.
    """
    points = []
    attrs = []
    
    for idx, row in gdf.iterrows():
        geom = row.geometry
        
        if geom.geom_type == 'LineString':
            coords = list(geom.coords)
            for i in indices:
                if -len(coords) <= i < len(coords):
                    point = Point(coords[i])
                    points.append(point)
                    attrs.append({**row.to_dict(), 'vertex_index': i})
        
        elif geom.geom_type == 'MultiLineString':
            for line in geom.geoms:
                coords = list(line.coords)
                for i in indices:
                    if -len(coords) <= i < len(coords):
                        point = Point(coords[i])
                        points.append(point)
                        attrs.append({**row.to_dict(), 'vertex_index': i})
    
    return gpd.GeoDataFrame(attrs, geometry=points, crs=gdf.crs)

#To find the intersection between two lines
def find_line_intersections(gdf1, gdf2, crs=None):
    """
    Find intersection points between two layers of lines.
    
    Parameters:
    - gdf1, gdf2: GeoDataFrames with LineString/MultiLineString geometries.
    - crs: Coordinate reference system (if not defined).
    
    Returns:
    - GeoDataFrame with intersection points and attributes from both lines.
    """
    if crs is None:
        crs = gdf1.crs
    gdf1 = gdf1.to_crs(crs)
    gdf2 = gdf2.to_crs(crs)
    
    spatial_index = gdf2.sindex
    
    intersecciones = []
    for idx1, linea1 in gdf1.iterrows():
        # Finding candidates using the spatial index
        posibles_matches = list(spatial_index.intersection(linea1.geometry.bounds))
        for idx2 in posibles_matches:
            linea2 = gdf2.iloc[idx2]
            # Calculate intersection
            intersect = linea1.geometry.intersection(linea2.geometry)
            if not intersect.is_empty:
                # Managing MultiPoint and Point
                if intersect.geom_type == "MultiPoint":
                    puntos = list(intersect.geoms)
                else:
                    puntos = [intersect]
                # Save attributes for both lines
                for punto in puntos:
                    datos = {
                        **{f"gdf1_{k}": v for k, v in linea1.items()},
                        **{f"gdf2_{k}": v for k, v in linea2.items()},
                        "geometry": punto
                    }
                    intersecciones.append(datos)
    
    return gpd.GeoDataFrame(intersecciones, crs=crs)
    
#To see if they are equal points
def son_puntos_iguales(gdf1, gdf2):
    """
    Compare two GeoDataFrames of point geometries based on their coordinates.
    
    Parameters:
    - gdf1: A GeoDataFrame containing Point geometries.
    - gdf2: A GeoDataFrame containing Point geometries.
    
    Returns:
    - A boolean value indicating whether both GeoDataFrames contain exactly the
      same set of point coordinates.
    """
    coords1 = set((p.x, p.y) for p in gdf1.geometry)
    coords2 = set((p.x, p.y) for p in gdf2.geometry)
    return coords1 == coords2

#To find the difference between vectors
def vector_difference(input_gdf, overlay_gdf):
    """
    Calculates the geometric difference between two vector layers.
    
    Parameters:
    - input_gdf: GeoDataFrame with geometries to which the difference will be applied.
    - overlay_gdf: GeoDataFrame with geometries that will be subtracted from `input_gdf`.
    
    Returns:
    - GeoDataFrame with the parts of `input_gdf` that do not intersect with `overlay_gdf`.
    """
    # Ensure same CRS
    input_gdf = input_gdf.to_crs(overlay_gdf.crs)
    
    # Make a spatial difference
    diferencia = gpd.overlay(
        input_gdf,
        overlay_gdf,
        how='difference',
        keep_geom_type=True 
    )
    return diferencia

#To find the first element downstream
def encontrar_primer_elemento(punto, drenaje, target_layer, grafo_abajo):
    """
    Find the first feature in a target layer encountered when traversing a drainage
    network downstream from a given point.
    
    Parameters:
    - punto: A GeoSeries or GeoDataFrame row containing a Point geometry that defines
      the starting location.
    - drenaje: A GeoDataFrame representing the drainage network, where each row
      corresponds to a line segment.
    - target_layer: A GeoDataFrame containing geometries to be searched for
      intersection with the drainage network.
    - grafo_abajo: A dictionary or mapping structure that defines downstream
      connectivity between drainage segments, where keys are segment indices and
      values are lists of downstream segment indices.
    
    Returns:
    - The first row (as a GeoSeries) from the target_layer that intersects any
      drainage segment encountered during the downstream traversal.
    - Returns None if no intersecting element is found.
    """
    #Find initial segment
    distancias = drenaje.geometry.distance(punto.geometry)
    idx_segmento = distancias.idxmin()
    
    # Downstream search
    segmentos_visitados = set()
    cola = [idx_segmento]
    
    while cola:
        idx_actual = cola.pop(0)
        if idx_actual in segmentos_visitados:
            continue
        segmentos_visitados.add(idx_actual)
        
        segmento = drenaje.loc[idx_actual]
        
        # Buscar intersección con target_layer
        interseccion = target_layer[target_layer.intersects(segmento.geometry)]
        if not interseccion.empty:
            return interseccion.iloc[0]  # First item found
        
        cola.extend(grafo_abajo[idx_actual])
    
    return None

drenaje_global = None
poligonos_global = None
buffer_manan_global = None
grafo_abajo_global = None

def procesar_punto(idx_punto):
    """
    Process a single point by locating the first downstream intersecting features
    in two reference layers, extracting selected attributes, and (when possible)
    retrieving a hydraulic head value using a cell identifier.
    
    Parameters:
    - idx_punto: An index value used to select one row from the global GeoDataFrame
      'puntos'.
    
    Returns:
    - A dictionary with the following keys:
        - 'id_punto': An identifier for the point (as coded, this uses the variable
          'idx'; see Notes).
        - 'name': The 'name' attribute from the first downstream intersecting feature
          found in 'buffer_manan_global', or None if not found.
        - 'elev_manan': The 'elev_manan' attribute from the same feature, or None if
          not found.
        - 'elev_drn': A head value extracted from the global array 'head' using a
          (layer, row, col) tuple derived from 'cellid', or None if not obtained.
        - 'dif': The difference elev_drn - elev_manan when both values are available,
          otherwise None.
        - 'geometry': The geometry of the selected point.
    """
    punto = puntos.loc[idx_punto]
    celda_drn = encontrar_primer_elemento(punto, drenaje_global, poligonos_global, grafo_abajo_global)
    buffer = encontrar_primer_elemento(punto, drenaje_global, buffer_manan_global, grafo_abajo_global)
    if buffer is not None:
        name = buffer["name"]
        elev_manan = buffer["elev_manan"]
    else:
        name = None
        elev_manan = None
    elev_drn = None
    # Obtain hydraulic load from cellid
    if celda_drn is not None and "cellid" in celda_drn:
        cellid_raw = celda_drn["cellid"]

        if isinstance(cellid_raw, str):
            import ast
            try:
                cellid = ast.literal_eval(cellid_raw)
            except Exception as e:
                print(f"Error al convertir cellid: {cellid_raw}")
                cellid = None
        else:
            cellid = cellid_raw

        if isinstance(cellid, tuple) and len(cellid) == 3:
            layer, row, col = cellid
            try:
                elev_drn = head[layer, row, col]
            except IndexError:
                print(f"indice fuera de rango: {cellid}")
        else:
            print(f"cellid mal formado: {cellid_raw}")
    else:
        print(f"celda_drn invalido para idx {idx_punto}")
    dif = elev_drn - elev_manan if (elev_manan is not None and elev_drn is not None) else None
    return {
        "id_punto": idx,
        "name": name,
        "elev_manan": elev_manan,
        "elev_drn": elev_drn,
        "dif": dif,
        "geometry": punto.geometry}

In [ ]:
buf_manan=os.path.join(carpeta, 'buf_manan.shp') #spring buffer for geospatial analysis
puntos_finales_red=os.path.join(carpeta, 'puntos_finales_red.shp') #end points in drainage network from each spring
stream_manan_final_inters=os.path.join(carpeta, 'stream_manan_final_inters.shp') # drainage cut off
    

In [12]:
#Define Output

snapped_points = carpeta+'/Man_ajust.shp'#Springs connected to the drainage network
flow_paths = carpeta+'/ruta_flujo.tif' #is a tif of drainage from the adjusted springs
flow_paths_lines = carpeta+'/ruta_flujo_lines.shp' # drainage polyline from adjusted springs
Drn_celdas_total= gdf_domain # polygon of cells used for drainage in the domain, change every time of you change the cell size
Drn_celdas_activas= gdf_drn_activas # polygon with active cells
puntos_final_v1=carpeta+'/ptos_linea_fin1.shp'  #points at the end of the drainage network created from tif
manan_filt_dom= carpeta+'/manan_filt_dom.shp' #springs adjusted to this new drainage system
snap_point_temp=carpeta+'/snap_point_temp.shp' # temporary point snaps
manan_filt_dom_2=carpeta+'/manan_filt_dom_2.shp' #springs that actually work in this dataset of springs
buf_manan=carpeta+'/buf_manan.shp' #spring buffer for geospatial analysis
puntos_finales_red=carpeta+'/puntos_finales_red.shp' #end points in drainage network  from each spring
flow_paths_2 = carpeta+'/ruta_flujo_2.tif' #is a tif of the drainage from the end points
stream_manan_final=carpeta+'/stream_manan_final.shp' #drainage from endpoints in shape
stream_manan_final_inters=carpeta+'/stream_manan_final_inters.shp'  # drainage cut off
resultados_final=carpeta+'/resultados_final.shp' # shape with points at the end of the drains with information on active drains that touch with their piezometry and spring elevation
resultados_filtrados=carpeta+'/resultados/resultados_filtrados.shp' # filtered final

In [13]:
#Use Saga to connect occupied springs to stream polyline
saga = SAGA(r"C:\Program Files\SAGA\saga_cmd.exe")
# Choose library
preprocessor = saga / 'shapes_points'
# Choose the tool
Snap_point_lines = preprocessor / 'Snap Points to Lines'
Snp=Snap_point_lines.execute(input=original_points,snap=drainage_network,output=snapped_points ) 

In [14]:
#Create raster and polyline streams from springs
wbt = whitebox.WhiteboxTools()
wbt.set_verbose_mode(True)
output_flow = flow_paths
wbt.trace_downslope_flowpaths(snapped_points, FP,output=flow_paths)  
wbt.raster_streams_to_vector(flow_paths,FP,output=flow_paths_lines) 
fpl = gpd.read_file(flow_paths_lines)
fpl=fpl.set_crs("EPSG:32719") #Just to confirm
fpl.to_file(flow_paths_lines) 

.\whitebox_tools.exe --run="TraceDownslopeFlowpaths" --seed_pts='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\Calib_Manan_Sim_12.136/Man_ajust.shp' --d8_pntr='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\FP.tif' --output='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\Calib_Manan_Sim_12.136/ruta_flujo.tif' -v --compress_rasters=False

**************************************
* Welcome to TraceDownslopeFlowpaths *
* Powered by WhiteboxTools           *
* www.whiteboxgeo.com                *
**************************************
Reading flow direction data...
Locating seed points: 0%
Locating seed points: 1%
Locating seed points: 2%
Locating seed points: 3%
Locating seed points: 4%
Locating seed points: 5%
Locating seed points: 6%
Locating seed points: 7%
Locating seed points: 8%
Locating seed points: 9%
Locating seed points: 10%
Locating seed points: 11%
Locating seed points: 12%
Locating seed points: 13%
Locating seed points: 14%
Locating seed points: 15%
Loc

In [69]:
# Load the previous polyline file
gdf = gpd.read_file(flow_paths_lines)
#Define formula for extract endpoints
def extraer_puntos_finales(geom):
    """
    Extract the start and end points from a LineString or MultiLineString geometry.
    
    Parameters:
    - geom: A Shapely geometry of type LineString or MultiLineString.
    
    Returns:
    - A list of Point geometries corresponding to the first and last coordinate
      of each LineString contained in the input geometry.
    """
    puntos = []
    if geom.geom_type == 'MultiLineString':
        for linea in geom.geoms:
            if len(linea.coords) >= 2:
                puntos.extend([Point(linea.coords[0]), Point(linea.coords[-1])])
    elif geom.geom_type == 'LineString':
        if len(geom.coords) >= 2:
            puntos.extend([Point(geom.coords[0]), Point(geom.coords[-1])])
    return puntos

# Step 1: Extract all endpoints
puntos_finales = []
for idx, fila in gdf.iterrows():
    puntos = extraer_puntos_finales(fila.geometry)
    for punto in puntos:
        puntos_finales.append({
            'id_linea': idx,
            'tipo_punto': 'inicio' if punto.equals(puntos[0]) else 'fin',
            'geometry': punto
        })

# Step 2: Identify intersection points (which appear on more than one line)
precision = 3  # Adjust according to coordinate system (e.g., 3 decimal places for UTM)
contador_puntos = defaultdict(int)

for p in puntos_finales:
    coord = (round(p['geometry'].x, precision), 
             round(p['geometry'].y, precision))
    contador_puntos[coord] += 1

# Step 3: Filter unique points (no intersections)
puntos_filtrados = [
    p for p in puntos_finales 
    if contador_puntos[(round(p['geometry'].x, precision), 
                       round(p['geometry'].y, precision))] == 1
]

# Create and save the result
gdf_puntos_filtrados = gpd.GeoDataFrame(puntos_filtrados, crs=gdf.crs)
gdf_puntos_filtrados.to_file(puntos_final_v1) 

#Load data
puntos = gpd.read_file(puntos_final_v1)
dominio=gpd.read_file(dom)

#Filter those within the domain to eliminate the lowest points
puntos_dentro = gpd.sjoin(
    puntos,
    dominio,
    how="inner",
    predicate="within"
).drop_duplicates(subset=['geometry'])
puntos_dentro.to_file(manan_filt_dom)

capa_entrada_0 = gpd.read_file(manan_filt_dom)  # Filter layer

#Create a buffer to intersect manan_filt_dom and extract the value of the “name” from the column 
#(but first create a point-to-point snap so that the points actually overlap due to position differences)
Snap_point_point = preprocessor / 'Snap Points to Points'
Snap_point_point.execute(input=snapped_points,snap=manan_filt_dom,output=snap_point_temp, distance=20) #20 because the snapped points we use come from a raster where the largest vector at the center of the rectangle does not exceed 20 m.
capa_dato=gpd.read_file(snap_point_temp)
capa_dato.geometry = capa_dato.geometry.buffer(1)
capa_dato=capa_dato.set_crs(dominio.crs)
puntos_unidos = gpd.sjoin(
    capa_entrada_0,
    capa_dato[["name","altitude", "geometry"]],  # Select the column to extract + geometry
    how="left",  
    predicate="intersects", 
    lsuffix="_orig",
    rsuffix="_datos"
)

# clean columns
puntos_unidos = puntos_unidos.drop(columns=['index_datos'], errors='ignore')
puntos_unidos=puntos_unidos.drop_duplicates(subset="id_linea", keep="first")
puntos_unidos = puntos_unidos.rename(columns={"altitude": "elev_manan"})
capa_entrada=puntos_unidos.drop(columns=["tipo_punto","id","id_linea"])

# Prepare reference layer
capa_referencia = Drn_celdas_total  # reference file
capa_referencia = capa_referencia.drop(columns=['index_right', 'index_left'], errors='ignore')

resultado = gpd.sjoin(capa_entrada, capa_referencia,  predicate="intersects")
resultado=resultado.rename(columns={'elev_drn':'elev_manan'})
resultado=resultado.drop(columns=["index_righ"])
resultado.to_file(manan_filt_dom_2)
buf = resultado.copy()
buf.geometry = buf.geometry.buffer(2)
buf=buf.set_crs(dominio.crs)
buf.to_file(buf_manan)

C:\Users\nick_\AppData\Local\Temp\ipykernel_8736\3080130842.py:67: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  puntos_dentro.to_file(manan_filt_dom)
C:\Users\nick_\anaconda3\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D Point' is converted to 'Point Z'
  return ogr_read(
C:\Users\nick_\AppData\Local\Temp\ipykernel_8736\3080130842.py:100: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  resultado.to_file(manan_filt_dom_2)
C:\Users\nick_\AppData\Local\Temp\ipykernel_8736\3080130842.py:104: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  buf.to_file(buf_manan)


In [75]:
#Extract drainage start points

t0 = time.time()
wbt = whitebox.WhiteboxTools()
wbt.work_dir = carpeta

# 1. Load initial data
stream_net_real = gpd.read_file(drainage_network)
manan_filt = gpd.read_file(manan_filt_dom_2)
stream_for_manan = gpd.read_file(flow_paths_lines)

if stream_for_manan.crs is None:
    stream_for_manan = stream_for_manan.set_crs(epgs)
if manan_filt.crs is None:
    manan_filt = manan_filt.set_crs(epgs)
if stream_net_real.crs is None:
    stream_net_real = stream_net_real.set_crs(epgs)

stream_for_manan.to_file(carpeta+'/temp_union_-1.shp')

# 2. Prepare the spatial index
sindex = stream_net_real.sindex

max_iteraciones = 20
tolerancia = 0.1
union_prev = None
vertices_prev = None

for i in range(max_iteraciones):
    print(f"\n--- Iteration {i+1} ---")
    
    # A. 1 m buffer with vectorized operation
    manan_filt["buffer"] = manan_filt.geometry.buffer(1.0)
    buf1 = gpd.GeoDataFrame(geometry=manan_filt["buffer"], crs=manan_filt.crs)
    buffer_union = buf1.geometry.union_all()

    # B. Optimized spatial selection with index
    # Just search for streams that may intersect with any buffer using the spatial index.
    possible_matches_index = list(sindex.intersection(buffer_union.bounds))
    possible_matches = stream_net_real.iloc[possible_matches_index]
    selec = possible_matches[possible_matches.intersects(buffer_union)]

    # C. Perform a WhiteboxTools union operation using temporary files on disk
    temp_union = f"temp_union_{i}.shp"
    if selec.crs is None:
        selec = selec.set_crs(epgs)
    selec.to_file(carpeta+f'/temp_selec_{i}.shp')
    selec_path = f"temp_selec_{i}.shp"
    prev_union_path = f"temp_union_{i-1}.shp" if i > 0 else carpeta+'/temp_union_-1.shp'
    wbt.union(prev_union_path, selec_path, temp_union)
    
    # D. Extract vertices and calculate intersections
    selec_temp = gpd.read_file(carpeta+f'/temp_selec_{i}.shp')
    if selec_temp.crs is None:
        selec_temp = selec_temp.set_crs(epgs)    
    nuevos_vertices = extract_specific_vertices(selec_temp, indices=[0])
    
    nuevas_intersecciones = find_line_intersections(selec_temp, selec_temp)
    nuevas_intersecciones = nuevas_intersecciones[nuevas_intersecciones.geometry.type.isin(["Point", "MultiPoint"])].explode(index_parts=True)
    nuevas_intersecciones["buffer"] = nuevas_intersecciones.geometry.buffer(1.0)
    buf2 = gpd.GeoDataFrame(geometry=nuevas_intersecciones["buffer"], crs=manan_filt.crs)
    dif = vector_difference(nuevos_vertices, buf2)
    nuevos_vertices = dif

    # F. Stop condition
    if i > 0 and son_puntos_iguales(nuevos_vertices, vertices_prev):
        print("Identical points! Stopping loop.")
        break
        
    # G. Update for the next iteration
    vertices_prev = nuevos_vertices.copy()
    manan_filt = nuevos_vertices

nuevos_vertices.to_file(puntos_finales_red)

print(f"\n---Optimized process completed  ---\nTook {time.time() - t0:.2f} seconds")



--- Iteration 1 ---
.\whitebox_tools.exe --run="Union" --wd="C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\Calib_Manan_Sim_12.136" --input='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\Calib_Manan_Sim_12.136/temp_union_-1.shp' --overlay='temp_selec_0.shp' --output='temp_union_0.shp' --snap=0.0 -v --compress_rasters=False

****************************
* Welcome to Union         *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 33%
Progress: 34%
Progress: 35%
P

C:\Users\nick_\AppData\Local\Temp\ipykernel_8736\696059023.py:74: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nuevos_vertices.to_file(puntos_finales_red)


In [76]:
#Crear drenaje desde puntos finales de manera mas depurada.
wbt = whitebox.WhiteboxTools()
wbt.set_verbose_mode(True)
output_flow = flow_paths_2
wbt.trace_downslope_flowpaths(puntos_finales_red, FP,output=flow_paths_2) 
wbt.raster_streams_to_vector(flow_paths_2,FP,output=stream_manan_final) 
fpl2 = gpd.read_file(stream_manan_final)
fpl2=fpl2.set_crs(epgs)
fpl2.to_file(stream_manan_final)

.\whitebox_tools.exe --run="TraceDownslopeFlowpaths" --seed_pts='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\Calib_Manan_Sim_12.136/puntos_finales_red.shp' --d8_pntr='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\FP.tif' --output='C:\Users\nick_\OneDrive\Escritorio\Raster\calib_output\Calib_Manan_Sim_12.136/ruta_flujo_2.tif' -v --compress_rasters=False

**************************************
* Welcome to TraceDownslopeFlowpaths *
* Powered by WhiteboxTools           *
* www.whiteboxgeo.com                *
**************************************
Reading flow direction data...
Locating seed points: 0%
Locating seed points: 1%
Locating seed points: 2%
Locating seed points: 3%
Locating seed points: 4%
Locating seed points: 5%
Locating seed points: 6%
Locating seed points: 7%
Locating seed points: 8%
Locating seed points: 9%
Locating seed points: 10%
Locating seed points: 11%
Locating seed points: 12%
Locating seed points: 13%
Locating seed points: 14%
Locating seed poin

In [77]:
#Intersección para que funcion trabaje bien
drenaje = gpd.read_file(stream_manan_final)       
Dominio_celdas=gdf_domain 
interseccion1 = gpd.overlay(drenaje,Dominio_celdas, how="intersection")
interseccion_mono=interseccion1.explode(index_parts=False).reset_index(drop=True)
interseccion_mono.to_file(carpeta+'/stream_manan_final_inters.shp')

In [78]:
puntos = gpd.read_file(puntos_finales_red)   # Points of interest with data 
drenaje = gpd.read_file(stream_manan_final_inters)  # Drainage network (polylines)
poligonos = gdf_drn_activas      # Active cells to intersect
buffer_manan=gpd.read_file(buf_manan) #Buffer of springs dots

In [43]:
###Si ya corrieron todo esto antes, deberian haber solo corrido la primera celda indicada y luego lo que viene

In [79]:
t0 = time.time()
from tqdm import tqdm    
# Extract the end points from each line
end_points = drenaje.geometry.apply(lambda geom: Point(list(geom.coords)[-1]))
start_points = drenaje.geometry.apply(lambda geom: Point(list(geom.coords)[0]))
# Create a GeoDataFrame of beginnings
gdf_start = gpd.GeoDataFrame({'idx': drenaje.index, 'geometry': start_points}, crs=drenaje.crs)
sindex = gdf_start.sindex  # spatial index for beginnings
# For each line, find who it connects to downstream using the spatial index
tolerancia = 1  # 1 meter
grafo_abajo = defaultdict(list)
for idx, end_pt in zip(drenaje.index, end_points):
    # Find potential matches using the spatial index
    posibles = list(sindex.intersection(end_pt.buffer(tolerancia).bounds))
    for idx2 in posibles:
        start_pt = gdf_start.loc[idx2].geometry
        if end_pt.distance(start_pt) < tolerancia:
            grafo_abajo[idx].append(gdf_start.loc[idx2].idx)
# Assign to global variables
drenaje_global = drenaje
poligonos_global = poligonos
buffer_manan_global = buffer_manan
grafo_abajo_global = grafo_abajo
# List of indexes to process
indices = list(puntos.index)
resultados = []
for idx in tqdm(indices):
    resultado = procesar_punto(idx)
    resultados.append(resultado)
gdf_resultados = gpd.GeoDataFrame(resultados, crs=puntos.crs)
    
#Work on data to include as observations
#Delete rows with null values in ‘name’ or ‘elev_drn’ IF ANY
gdf_clean = gdf_resultados.dropna(subset=["name", "elev_drn"])
# Filter by maximum elev_drn (if there are ties, keep the first one)
gdf_filtrado = gdf_clean.loc[gdf_clean.groupby("name")["elev_drn"].idxmax()]
gdf_filtrado = gdf_filtrado.sort_values(["name", "elev_drn"], ascending=[True, False])
    
#Extract obs
elev=gdf_filtrado[["name","elev_drn"]].copy()
elev['time']=1.0
elev_pivoted = elev.pivot(index='time', columns='name', values='elev_drn')

calib_manan_final =os.path.join(calib_manan_input,'Results')
os.makedirs(calib_manan_final, exist_ok=True)
elev_pivoted.to_csv(os.path.join(calib_manan_final,f"Model_v{version}.Sim_elev.csv"))    
print(f"Demoro {time.time() - t0:.2f} segundos") #time
 

100%|██████████| 245/245 [00:15<00:00, 15.42it/s]


Demoro 19.52 segundos
